In [1]:
from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

# from pprint import pprint
# word = 'know'
# response = parser.fetch(word)
# pprint(response)

In [2]:
import re

def fetchWikiDict(word_list):
    Dict = {}
    count = 0
    for word in word_list:
        response = parser.fetch(word)
        for meaning in response:

            # filtering pronunciations.
            IPAs = {}
            for p in meaning['pronunciations']['text']:
                if 'IPA:' in p:
                    UK = re.search('England|UK|Received Pronunciation', p)
                    US = re.search('US|American|GenAm|enPR', p)
                    if US and 'US' not in IPAs:
                        IPAs['US'] = re.search('(IPA: )(.*)',p).group(2)
                    if UK and 'UK' not in IPAs:
                        IPAs['UK'] = re.search('(IPA: )(.*)',p).group(2)
                    if re.match('IPA:',p) and 'IPA' not in IPAs: 
                        IPAs['IPA'] = re.search('(IPA: )(.*)',p).group(2)
            if len(IPAs) == 3:
                del IPAs['IPA']
            if len(IPAs) == 2:
                if 'UK' in IPAs and 'US' in IPAs:
                    if IPAs['UK'] == IPAs['US']:
                        IPAs['IPA'] = IPAs['US']  
                        del IPAs['UK'], IPAs['US']
                    
            for definition in meaning['definitions']:
                word_class = definition['partOfSpeech']

                def_list = []
                count = 0
                for df in definition['text']:
                    count += 1
                    if count == 1:
                        morfology = df
                        continue
                    match = re.search("(?:\(.*)(UK|slang|obsolete|dialectal|Caribbean|MLE)(?:.*\))", df)
                    if match:
                        continue
                    match = re.match("Dated form", df)
                    if match:
                        continue
                    def_list.append(df)  

                if len(def_list) == 0: 
                    continue
                if len(definition["examples"]) == 0: 
                    continue
                
                examples = []
                for Eg in definition['examples']:
                    if len(Eg) < 60 and not re.match('Audio',Eg):
                        examples.append(Eg)
                    
                if len(examples) == 0: 
                    continue
                
                data = {}    
                data['class'] = word_class
                data['examples'] = examples
                    
                if word in Dict:       
                    Dict[word]['meanings'].append(data) 
                else: 
                    Dict[word] = {}
                    Dict[word]["morfology"] = morfology
                    Dict[word]['pronunciation'] = IPAs
                    Dict[word]['meanings'] = []
                    Dict[word]['meanings'].append(data) 

    return Dict

In [3]:
import json

with open("wordsList.json", 'r') as f:
    wordsList = json.loads(f.read())
    wikiData = fetchWikiDict(wordsList)
with open("wikiData.json", 'w') as f:
    json.dump(wikiData, f, ensure_ascii = False)